In [1007]:
import numpy as np
import math
import random
from functools import reduce

In [1008]:
def generateRandomVector(d):
    return np.random.randint(2, size = d)

In [1009]:
dimensions = 10000

A = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

In [1010]:
def similarity(A, B):
    if A.size != B.size:
        print('A and B have different dimensions.')
    
    count = 0
    for i in range(0,(A.size - 1)):
        if A[i]==B[i]:
            count += 1
    
    return count/A.size


In [1011]:
def calculateSTDs(A, B):
    return abs((similarity(A,B) * A.size  - 5000) / 50)

In [1012]:
def applyThreshold(V, n):
    if n % 2 == 1:
        threshold = lambda x: 1 if x > n/2 else 0
    else:
        threshold = lambda x: 1 if x > n/2 else (1 if x == n/2 and bool(random.getrandbits(1)) else 0)

    vectorized_threshold = np.vectorize(threshold)
    return vectorized_threshold(V)

In [1013]:
def arithmeticSum(A, B):
    sum = A + B

    return applyThreshold(sum, 2) 

In [1014]:
def arithmeticSubtraction(A, B):
    compliment = 1 - B

    return arithmeticSum(A, compliment)

In [1015]:
def arithmeticSumVectors(vectors):
    sum = reduce(lambda x, y: x+y, vectors)

    return applyThreshold(sum, len(vectors))

In [1016]:
mean = arithmeticSum(A, B)

In [1017]:
def comparison(A, B):
    dividend = sum(A*B)
    divisor = math.sqrt(sum(A**2) * sum(B**2))

    return dividend / divisor

In [1018]:
print('sim  A, A:         ', similarity(A,A))
print('sim  A, B:         ', similarity(A,B))
print('sim  A, mean:      ', similarity(A,mean))
print('sim  B, mean:      ', similarity(B,mean))

print('comp A, B:         ', comparison(A, B))
print('comp A, mean:      ', comparison(A, mean))
print('comp B, mean:      ', comparison(B, mean))

print('STDs A, B:         ', calculateSTDs(A, B))
print('STDs A, mean:     ', calculateSTDs(A, mean))
print('STDs B, mean:     ', calculateSTDs(B, mean))

print('sim  A, mean - B:  ',similarity(A, arithmeticSubtraction(mean, B)))
print('STDs A, mean - B: ',calculateSTDs(A, arithmeticSubtraction(mean, B)))

print('sim  B, mean - B:  ',similarity(B, arithmeticSubtraction(mean, B)))
print('STDs B, mean - B: ',calculateSTDs(B, arithmeticSubtraction(mean, B)))

sim  A, A:          0.9999
sim  A, B:          0.4929
sim  A, mean:       0.7446
sim  B, mean:       0.7482
comp A, B:          0.48460281706487074
comp A, mean:       0.7407356055339365
comp B, mean:       0.7447285498772511
STDs A, B:          1.42
STDs A, mean:      48.92
STDs B, mean:      49.64
sim  A, mean - B:   0.6263
STDs A, mean - B:  24.700000000000017
sim  B, mean - B:   0.367
STDs B, mean - B:  25.9


In [1019]:
def mult(A, B):
    return np.bitwise_xor(A, B)

In [1020]:
print(A)
print(B)
print(mult(A, B))
print(comparison(A, mult(mult(A, B), B)))

[1 0 0 ... 1 0 1]
[0 1 1 ... 1 1 0]
[1 1 1 ... 0 1 1]
1.0


In [1021]:
class ItemMemory:
    def __init__(self, vectors = []):
        self.vectors = vectors
    
    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanUp(self, V):
        return max(self.vectors, key=lambda x: comparison(V, x[1]))


In [1022]:
itemMemory = ItemMemory()
itemMemory.addVector('A', A)
itemMemory.addVector('B', B)
itemMemory.addVector('mean', mean)

In [1023]:
print(itemMemory.vectors)
print(itemMemory.cleanUp(B))

[('A', array([1, 0, 0, ..., 1, 0, 1])), ('B', array([0, 1, 1, ..., 1, 1, 0])), ('mean', array([0, 0, 1, ..., 1, 1, 1]))]
('B', array([0, 1, 1, ..., 1, 1, 0]))


In [1028]:
itemMemoryA = ItemMemory()

X = generateRandomVector(dimensions)
A = generateRandomVector(dimensions)

Y = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

Z = generateRandomVector(dimensions)
C = generateRandomVector(dimensions)

itemMemoryA.addVector('A', A)
itemMemoryA.addVector('B', B)
itemMemoryA.addVector('C', C)
itemMemoryA.addVector('X', X)
itemMemoryA.addVector('Y', Y)
itemMemoryA.addVector('Z', Z)

H = arithmeticSumVectors([mult(A, X), mult(B, Y), mult(C, Z)])

V = itemMemory.cleanUp(mult(H, X))

print(V[0])
print(similarity(mult(H, X), A))

A
0.749
